# Motor Loss Table Generation
This tool is used for motor loss table generation for GEN5SW

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import plotly.graph_objects as go # https://plotly.com/python/getting-started/
import seaborn as sns
import Utils
import datetime
import getpass
import csv
from scipy.interpolate import griddata

In [ ]:
# Setup
Project = 'Bowfell'
Motor = 'AML'
Date = datetime.datetime.now()
User = getpass.getuser()

Table_Size_x        = 26
Table_Size_y        = 17

Max_Motor_Torque    = 300
Max_Motor_Speed     = 13000

Torque_BP           = np.linspace(0, Max_Motor_Torque, Table_Size_x, dtype='int')
Speed_BP            = np.array([0, 500, 1000, 2000, 3000, 4000, 5000, 5800, 7200, 8300, 9000, 10000, 11000, 12000],  dtype='int')

path = r'/Users/Admin/Documents/work/Motor Loss Table Gen/Example Data/Bowfell Sample Data/q1'
#/Users/Admin/Documents/work/Motor Loss Table Gen/Example Data/Bowfell Sample Data/q1
#/Users/sfreedman/Desktop/Run2/q1
Dwell_Period = 500

In [ ]:
# Join path and look for any .csv's
all_files = glob.glob(os.path.join(path, "*Signals*.csv"))

# Create dataframe from all_files csv's
df = pd.concat( (pd.read_csv(f) for f in all_files), ignore_index=True) 

## Report Data

In [ ]:
#Create table with test data details.

data = [
        [str(Date)],
        [str(Project)],
        [str(Motor)],
        [str(User)],
        [str(Table_Size_x)],
        [str(Table_Size_y)],
        [str(Max_Motor_Torque)],
        [str(Max_Motor_Speed)],
        [str(Torque_BP)],
        [str(Speed_BP)],
        [str(path)]
       ]

headers = [
            "Date Report Produced:",
            "Project:", 
            "Motor:", 
            "User:",
            "Table size (x) :",
            "Table size (y) :",
            "Maximum motor torque (Nm) :",
            "Maximum motor speed (rpm) :",
            "Torque breakpoints (Nm) :",
            "Speed breakpoints (Nm) :",
            "Data Path :"
            ]

Report_Table = pd.DataFrame(data, headers)
pd.set_option('display.max_colwidth', 0)

display(Report_Table)

for files in all_files:
    print(files)

In [ ]:
#Rename  signals in dataframe to something readable.
df.rename(columns = {
                    ' TesOp_B.L2m_TarTrq_IOP'                   :'T_Demanded',
                    ' tesOutputData.L2mTes_EstTrq.val_IOP'      :'T_Estimated',
                    'Transducer_Torque_IOP'                     :'T_Measured',
                    'Transducer_Speed_IOP'                      :'N_Measured',
                    ' tesInputData.L2mPosSpdArb_RotorSpd_IOP'   :'N_Unit_rpm',
                    ' sensvdcOutputData.L2mSensVdc_Vdc.val_IOP' :'V_DC',
                    ' sensidcOutputData.L2mSensIdc_Idc.val_IOP' :'I_DC',
                    ' posSpdArbInputData.L2mSensVmt_Vd_IOP'     :'Ud_RMS',
                    ' posSpdArbInputData.L2mSensVmt_Vq_IOP'     :'Uq_RMS',
                    ' tesOutputData.L2mTes_Id.val_IOP'          :'Id_Peak',
                    ' tesOutputData.L2mTes_Iq.val_IOP'          :'Iq_Peak',
                    'InverterEfficiency_IOP'                    :'Loss_Inv_Unit'
                    }, 
                      inplace = True)

#Delete unwanted coloumns to speed up processing.
df.drop(columns = {
                    ' TesOp_B.L2m_TarTrq_MCP',
                    ' tesOutputData.L2mTes_EstTrq.val_MCP',
                    'Transducer_Torque_MCP','Transducer_Speed_MCP',
                    ' tesInputData.L2mPosSpdArb_RotorSpd_MCP',
                    ' sensvdcOutputData.L2mSensVdc_Vdc.val_MCP',
                    ' sensidcOutputData.L2mSensIdc_Idc.val_MCP',
                    ' posSpdArbInputData.L2mSensVmt_Vd_MCP',
                    ' posSpdArbInputData.L2mSensVmt_Vq_MCP',
                    ' tesOutputData.L2mTes_Id.val_MCP',
                    ' tesOutputData.L2mTes_Iq.val_MCP',
                    'InverterEfficiency_MCP', 
                    'E2E Value',
                    'timestamp'
                    }, 
                    inplace=True)


## Transient Removal

Make sure the transient removal process is removing the required data; when there is a step change and time taken until steady state is reached.

If this is not achieved, adjust the variable <code>Dwell_Period</code> at the top of the Notebook.

In [ ]:
df["Step_Change"]   = '0'
df["Step_Change"]   = df["T_Demanded"].diff()

Step_index = ( df.index[df['Step_Change'] != 0] - 1 )
Stop_index = Step_index + Dwell_Period

# Plot transient removal sample
transient_sample = df.iloc[Step_index[3]-250 : Stop_index[3]+250]

transient_plot = go.Figure()

transient_plot.add_trace(go.Scatter (  
                                    x       = transient_sample.index.values, 
                                    y       = transient_sample["T_Demanded"], 
                                    name    = "Demanded Torque (Nm)",
                                    hovertemplate = '%{y:.2f} Nm'
                        )           )

transient_plot.add_trace(go.Scatter (  
                                    x       = transient_sample.index.values, 
                                    y       = transient_sample["T_Estimated"],
                                    name    = "Estimated Torque (Nm)",
                                    hovertemplate = '%{y:.2f} Nm'
                        )           )

transient_plot.add_trace(go.Scatter (  
                                    x       = transient_sample.index.values, 
                                    y       = transient_sample["T_Measured"], 
                                    name    = "Measured Torque (Nm)",
                                    hovertemplate = '%{y:.2f} Nm'
                        )           )

transient_plot.update_layout    (   
                                title       = 'Transient Removal Example',
                                xaxis_title = 'Sample (N)',
                                yaxis_title = 'Torque (Nm)'
                                )

transient_plot.add_annotation   (
                                x           = Step_index[3],
                                y           = df["T_Demanded"][Step_index[3]],
                                xref        = "x",
                                yref        = "y",
                                text        = "Start of removal",
                                showarrow   = True,
                                font        = dict   (
                                                    family  = "Arial, monospace",
                                                    size    = 14,
                                                    color   = "#ffffff"
                                                    ),
                                align       = "center",
                                arrowhead   = 2,
                                arrowsize   = 1,
                                arrowwidth  = 2,
                                arrowcolor  = "#636363",
                                bordercolor = "#c7c7c7",
                                borderwidth = 2,
                                borderpad   = 4,
                                bgcolor     = "#ff7f0e",
                                )

transient_plot.add_annotation   (
                                x           = Stop_index[3],
                                y           = df["T_Demanded"][Stop_index[3]],
                                xref        = "x",
                                yref        = "y",
                                text        = "End of removal",
                                showarrow   = True,
                                font        = dict   (
                                                    family  = "Arial, monospace",
                                                    size    = 14,
                                                    color   = "#ffffff"
                                                    ),
                                align       = "center",
                                arrowhead   = 2,
                                arrowsize   = 1,
                                arrowwidth  = 2,
                                arrowcolor  = "#636363",
                                bordercolor = "#c7c7c7",
                                borderwidth = 2,
                                borderpad   = 4,
                                bgcolor     = "#ff7f0e",
                                )

transient_plot.update_layout    (
                                hovermode="x unified",
                                )

transient_plot.show()

#Transient Removal
delete_slice = np.array(0)

for x in range( len(Step_index) ):
    temp_slice = np.arange(Step_index[x], Stop_index[x])
    delete_slice = np.append(delete_slice, temp_slice)

delete_slice = abs(delete_slice)
df.drop(index = delete_slice);

In [ ]:
# Post process signals

# Mechanical
df["N_Measured_rads"]   = Utils.rpm_2_rads(df["N_Measured"])
df["P_Measured"]        = df["N_Measured_rads"] * df["T_Measured"]

# DC Link
df["P_DC"]              = df["V_DC"] * df["I_DC"]

# Phase Peak 
df["Ud_Peak"]           = Utils.peak_2_rms(df["Ud_RMS"]) 
df["Uq_Peak"]           = Utils.peak_2_rms(df["Uq_RMS"]) 
  
# Phase RMS
df["Id_RMS"]            = Utils.peak_2_rms(df["Id_Peak"]) 
df["Iq_RMS"]            = Utils.peak_2_rms(df["Iq_Peak"]) 
  
# Phase
df["Is_RMS"]            = np.sqrt( ( df["Id_RMS"] * df["Id_RMS"] ) + ( df["Iq_RMS"] * df["Iq_RMS"] ) )
inverse_clarke_values   = Utils.inverse_clarke(df["Id_RMS"],df["Iq_RMS"])
df["I_a"]               = inverse_clarke_values[0]
df["I_b"]               = inverse_clarke_values[1]
df["I_c"]               = inverse_clarke_values[2]
df["P_Ph"]              = (3/2) * ( df["Ud_Peak"] * df["Id_Peak"] ) + ( df["Uq_Peak"] * df["Iq_Peak"] ) 

# Losses
df["Loss_Inv_Real"]     = df["P_DC"] - df["P_Ph"]
df["Loss_Mtr"]          = df["P_Ph"] - df["P_Measured"]
df["Loss_Mtr_Comp"]     = df["P_DC"] - df["Loss_Inv_Unit"] - df["P_Measured"]
df["Loss_Sys"]          = df["P_DC"] - df["P_Measured"]


In [ ]:
# ## Plot test points

# test_points_plot = go.Figure()

# test_points_plot.add_trace(go.Scatter   (  
#                                         x               = df["N_Measured"].round(-2), 
#                                         y               = df["T_Demanded"].round(-1), 
#                                         name            = "Motor Losses (W)",
#                                         mode            = "markers",
#                                         marker_symbol   = 'x'
#                             )           )

# test_points_plot.update_layout  (   
#                                 title       ='Test Points',
#                                 xaxis_title ='Speed (rpm)',
#                                 yaxis_title ='Torque (Nm)',
#                                 xaxis       = dict  (
#                                                     tickmode = 'linear',
#                                                     tick0 = 0,
#                                                     dtick = 1000
#                                                     )
#                                 )

# test_points_plot.show()

## Power and losses

In [ ]:
# Plot Losses
loss_plot = go.Figure()

loss_plot.add_trace(go.Scatter  (  
                                x       = df.index.values, 
                                y       = df["Loss_Mtr"], 
                                name    = "Motor Losses (W)",
                                hovertemplate = '%{y:.2f} W'
                    )           )

loss_plot.add_trace(go.Scatter  (  
                                x       = df.index.values, 
                                y       = df["Loss_Mtr_Comp"], 
                                name    = "Motor Losses Compensated (W)",
                                hovertemplate = '%{y:.2f} W'
                    )           )

loss_plot.add_trace(go.Scatter  (  
                                x       = df.index.values, 
                                y       = df["Loss_Inv_Real"], 
                                name    = "Inverter Calcualted Losses (W)",
                                hovertemplate = '%{y:.2f} W'
                    )           )

loss_plot.add_trace(go.Scatter  (  
                                x       = df.index.values, 
                                y       = df["Loss_Inv_Unit"], 
                                name    = "Inverter Losses (W)",
                                hovertemplate = '%{y:.2f} W'
                    )           )

loss_plot.add_trace(go.Scatter  (  
                                x       = df.index.values, 
                                y       = df["Loss_Sys"], 
                                name    = "System Losses (W)",
                                hovertemplate = '%{y:.2f} W'
                    )           )

loss_plot.update_layout (   
                        title       ='Losses',
                        xaxis_title ='Sample (N)',
                        yaxis_title ='Losses (W)',
                        hovermode   ="x unified",
                        hoverlabel_align = 'right',
                        template="plotly_white"
                        )

loss_plot.show()

In [ ]:
# Plot Powers
powers_plot = go.Figure()

powers_plot.add_trace(go.Scatter    (  
                                    x       = df.index.values, 
                                    y       = df["P_DC"], 
                                    name    = "DC Link Power (W)",
                                    hovertemplate = '%{y:.2f} W'
                        )           )

powers_plot.add_trace(go.Scatter    (  
                                    x       = df.index.values, 
                                    y       = df["P_Ph"],
                                    name    = "Phase Power (W)",
                                    hovertemplate = '%{y:.2f} W' 
                        )           )

powers_plot.add_trace(go.Scatter    (  
                                    x       = df.index.values, 
                                    y       = df["P_Measured"], 
                                    name    = "Mechanical Power (W)",
                                    hovertemplate = '%{y:.2f} W'
                        )           )

powers_plot.update_layout   (   
                            title       ='Powers',
                            xaxis_title ='Sample (N)',
                            yaxis_title ='Power (W)',
                            hovermode   ="x unified",
                            template="plotly_white"
                            )

powers_plot.show()

In [ ]:
Speed_Threshold     = 0
Torque_Threshold    = 0

# Determine operating quadrant
df["Operating_Quadrant"] = 'VEH_STP'
df["Operating_Quadrant"] = np.where( (df["N_Measured"] > Speed_Threshold) & (df["T_Measured"] > Torque_Threshold), 'DRV_FWD', df["Operating_Quadrant"] )
df["Operating_Quadrant"] = np.where( (df["N_Measured"] > Speed_Threshold) & (df["T_Measured"] < Torque_Threshold), 'BRK_FWD', df["Operating_Quadrant"] )
df["Operating_Quadrant"] = np.where( (df["N_Measured"] < Speed_Threshold) & (df["T_Measured"] < Torque_Threshold), 'DRV_REV', df["Operating_Quadrant"] )
df["Operating_Quadrant"] = np.where( (df["N_Measured"] < Speed_Threshold) & (df["T_Measured"] > Torque_Threshold), 'BRK_REV', df["Operating_Quadrant"] )

# Inverter Efficiency
df["Eff_Inv"] = 0.0
DRV_Index = np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') )
BRK_Index = np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') )
df["Eff_Inv"].iloc[DRV_Index] = Utils.eff_pc( df["P_Ph"].iloc[DRV_Index], df["P_DC"].iloc[DRV_Index] )
df["Eff_Inv"].iloc[BRK_Index] = Utils.eff_pc( df["P_DC"].iloc[BRK_Index], df["P_Ph"].iloc[BRK_Index] )

# Motor Efficiency
df["Eff_Mtr"] = 0.0
DRV_Index = np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') )
BRK_Index = np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') )
df["Eff_Mtr"].iloc[DRV_Index] = Utils.eff_pc( df["P_Measured"].iloc[DRV_Index], df["P_Ph"].iloc[DRV_Index] )
df["Eff_Mtr"].iloc[BRK_Index] = Utils.eff_pc( df["P_Ph"].iloc[BRK_Index], df["P_Measured"].iloc[BRK_Index] )

# System Efficiency
df["Eff_Sys"] = 0.0
DRV_Index = np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') )
BRK_Index = np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') )
df["Eff_Sys"].iloc[DRV_Index] = Utils.eff_pc( df["P_Measured"].iloc[DRV_Index], df["P_DC"].iloc[DRV_Index] )
df["Eff_Sys"].iloc[BRK_Index] = Utils.eff_pc( df["P_DC"].iloc[BRK_Index], df["P_Measured"].iloc[BRK_Index] )

In [ ]:
# Round measured speed to the nearest 100 rpm.
df["N_Measured_Rounded"] = df["N_Measured"].round(-2)

# Group the data in the dataframe by the measured speed (rounded) and torque demanded.
# Get average of all data within those subgroups and create new dataframe, df_steady_state.
df_steady_state = df.groupby(["N_Measured_Rounded", "T_Demanded"], as_index=False).agg("mean")

In [ ]:
Meshgrid_Speed_Array  = np.linspace(0, Max_Motor_Speed+1, Max_Motor_Speed+1)
Meshgrid_Torque_Array = np.linspace(0, Max_Motor_Torque+1, Max_Motor_Torque+1)

Meshgrid_Speed, Meshgrid_Torque = np.meshgrid(Meshgrid_Speed_Array, Meshgrid_Torque_Array)

# Determine operating quadrant
df_steady_state["Operating_Quadrant"] = 'VEH_STP'
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] > Speed_Threshold) & (df_steady_state["T_Measured"] > Torque_Threshold), 'DRV_FWD', df_steady_state["Operating_Quadrant"])
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] > Speed_Threshold) & (df_steady_state["T_Measured"] < Torque_Threshold), 'BRK_FWD', df_steady_state["Operating_Quadrant"])
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] < Speed_Threshold) & (df_steady_state["T_Measured"] < Torque_Threshold), 'DRV_REV', df_steady_state["Operating_Quadrant"])
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] < Speed_Threshold) & (df_steady_state["T_Measured"] > Torque_Threshold), 'BRK_REV', df_steady_state["Operating_Quadrant"])

# Rearrange data depending on quadrant.
if (df_steady_state["Operating_Quadrant"].any() == 'BRK_REV'):
    Meshgrid_Speed  = Meshgrid_Speed    *-1
    Speed_BP        = Speed_BP          *-1

elif (df_steady_state["Operating_Quadrant"].any() == 'BRK_FWD'):
    Meshgrid_Torque = Meshgrid_Torque   *-1
    Torque_BP       = Torque_BP         *-1
elif (df_steady_state["Operating_Quadrant"].any() == 'DRV_REV'):
    Meshgrid_Torque = Meshgrid_Torque   *-1
    Torque_BP       = Torque_BP         *-1
    Meshgrid_Speed  = Meshgrid_Speed    *-1
    Speed_BP        = Speed_BP          *-1

else:
    pass

In [ ]:
# Gird data

# Inverter losses
Loss_Inv_Grid       = griddata( df_steady_state.loc[ :, ['N_Measured_Rounded','T_Measured'] ], df_steady_state["Loss_Inv_Real"], (Meshgrid_Speed, Meshgrid_Torque), method="linear" )
Loss_Inv_Table      = Loss_Inv_Grid[:, Speed_BP]
Loss_Inv_Table      = Loss_Inv_Table[Torque_BP, :]

# Motor losses
Loss_Mtr_Grid       = griddata( df_steady_state.loc[ :, ['N_Measured_Rounded','T_Measured'] ], df_steady_state["Loss_Mtr_Comp"], (Meshgrid_Speed, Meshgrid_Torque), method="linear" )
Loss_Mtr_Table      = Loss_Mtr_Grid[:, Speed_BP]
Loss_Mtr_Table      = Loss_Mtr_Table[Torque_BP, :]

# System losses
Loss_Sys_Grid       = griddata( df_steady_state.loc[ :, ['N_Measured_Rounded','T_Measured'] ], df_steady_state["Loss_Sys"], (Meshgrid_Speed, Meshgrid_Torque), method="linear" )
Loss_Sys_Table      = Loss_Sys_Grid[:, Speed_BP]
Loss_Sys_Table      = Loss_Sys_Table[Torque_BP, :]

# Inverter efficiency
Eff_Inv_Grid       = griddata( df_steady_state.loc[ :, ['N_Measured_Rounded','T_Measured'] ], df_steady_state["Eff_Inv"], (Meshgrid_Speed, Meshgrid_Torque), method="linear" )
Eff_Inv_Table      = Eff_Inv_Grid[:, Speed_BP]
Eff_Inv_Table      = Eff_Inv_Table[Torque_BP, :]

# Motor efficiency
Eff_Mtr_Grid       = griddata( df_steady_state.loc[ :, ['N_Measured_Rounded','T_Measured'] ], df_steady_state["Eff_Mtr"], (Meshgrid_Speed, Meshgrid_Torque), method="linear" )
Eff_Mtr_Table      = Eff_Mtr_Grid[:, Speed_BP]
Eff_Mtr_Table      = Eff_Mtr_Table[Torque_BP, :]

# System efficiency
Eff_Sys_Grid       = griddata( df_steady_state.loc[ :, ['N_Measured_Rounded','T_Measured'] ], df_steady_state["Eff_Sys"], (Meshgrid_Speed, Meshgrid_Torque), method="linear" )
Eff_Sys_Table      = Eff_Sys_Grid[:, Speed_BP]
Eff_Sys_Table      = Eff_Sys_Table[Torque_BP, :]

In [ ]:
# Plot Contour of motor losses before interpolation
Loss_Inv_Plot = go.Figure()

Loss_Inv_Plot.add_trace(go.Contour  (
                                    z           = Loss_Inv_Table,
                                    x           = Speed_BP, 
                                    y           = Torque_BP,
                                    hovertemplate = 'Speed: %{x:.2f} rpm' + 
                                                    '<br>Torque: %{y:.2f} Nm</br>' +
                                                    'Inverter Losses: %{z:.2f} W',
                                    contours    = dict  (
                                                        coloring    ='heatmap',
                                                        showlabels  = True,
                                                        labelfont   = dict  (
                                                                            size = 10,
                                                                            color = 'white',
                                                                            )
                                                        ),               

                                    colorbar    = dict  (
                                                        title       = 'Inverter Losses (W)', 
                                                        titleside   = 'right',
                                                        titlefont   = dict  (
                                                                            size=12,
                                                                            family='Arial, sans-serif'
                                                                            )
                                                        )
                        )           )

Loss_Inv_Plot.update_layout (
                            title       = "Inverter Losses",
                            xaxis_title = "Speed (rpm)",
                            yaxis_title = "Torque (Nm)",
                            template="plotly_white"
                            )

Loss_Inv_Plot.show()

## Motor Power Losses
### Table of motor losses for defined torque and speed break points

In [ ]:
Loss_Mtr_Table_Display = pd.DataFrame(Loss_Mtr_Table)
Loss_Mtr_Table_Display.fillna(value = 0, inplace = True)
Loss_Mtr_Table_Display.columns = Speed_BP
Loss_Mtr_Table_Display.set_index(Torque_BP, inplace = True)

Loss_Mtr_Table_Display.style.background_gradient(cmap = sns.color_palette("dark:salmon_r", as_cmap=True) )

### Table of motor losses for defined torque and speed break points (Code Formatted)
The below table can be copied and pasted into the relevent C file relating the correct operating quadrant and voltage.

In [ ]:
Loss_Mtr_Table_Code = pd.DataFrame(Loss_Mtr_Table)

Loss_Mtr_Table_Code_Open = Loss_Mtr_Table_Code[0]
Loss_Mtr_Table_Code_Open = '{'

Loss_Mtr_Table_Code_Close = Loss_Mtr_Table_Code_Open
Loss_Mtr_Table_Code_Close = '},'

Loss_Mtr_Table_Code.loc[:,0:Loss_Mtr_Table_Code.shape[1]-2] = Loss_Mtr_Table_Code.applymap("{0:.2f},".format)

Loss_Mtr_Table_Code.insert(0, 0, Loss_Mtr_Table_Code_Open, allow_duplicates = True)

Loss_Mtr_Table_Code.columns = range(Loss_Mtr_Table_Code.shape[1])

Loss_Mtr_Table_Code.insert(Loss_Mtr_Table_Code.shape[1], Loss_Mtr_Table_Code.shape[1], Loss_Mtr_Table_Code_Close, allow_duplicates=True)

Loss_Mtr_Table_Code.loc[Loss_Mtr_Table_Code.shape[0]-1,Loss_Mtr_Table_Code.shape[1]-1] = '} '

Loss_Mtr_Table_Code = Loss_Mtr_Table_Code.to_string( index = False, header = False, justify = "right" )

print( Loss_Mtr_Table_Code )

### Contour plot of motor losses

In [ ]:
# Plot Contour of motor losses before interpolation
Loss_Mtr_Plot = go.Figure()

Loss_Mtr_Plot.add_trace(go.Contour  (
                                    z           = Loss_Mtr_Table,
                                    x           = Speed_BP, 
                                    y           = Torque_BP,
                                    hovertemplate = 'Speed: %{x:.2f} rpm' + 
                                                    '<br>Torque: %{y:.2f} Nm</br>' +
                                                    'Motor Losses: %{z:.2f} W',
                                    contours    = dict  (
                                                        coloring    ='heatmap',
                                                        showlabels  = True,
                                                        labelfont   = dict  (
                                                                            size = 10,
                                                                            color = 'white',
                                                                            )
                                                        ),               

                                    colorbar    = dict  (
                                                        title       = 'Motor Losses (W)', 
                                                        titleside   = 'right',
                                                        titlefont   = dict  (
                                                                            size=12,
                                                                            family='Arial, sans-serif'
                                                                            )
                                                        )
                        )           )

Loss_Mtr_Plot.update_layout (
                            title       = "Motor Power Losses",
                            xaxis_title = "Speed (rpm)",
                            yaxis_title = "Torque (Nm)",
                            template="plotly_white"
                            )

Loss_Mtr_Plot.show()

In [ ]:
# Plot Contour of system losses before interpolation
Loss_Sys_Plot = go.Figure()

Loss_Sys_Plot.add_trace(go.Contour  (
                                    z           = Loss_Sys_Table,
                                    x           = Speed_BP, 
                                    y           = Torque_BP,
                                    hovertemplate = 'Speed: %{x:.2f} rpm' + 
                                                    '<br>Torque: %{y:.2f} Nm</br>' +
                                                    'System Losses: %{z:.2f} W',
                                    contours    = dict  (
                                                        coloring    ='heatmap',
                                                        showlabels  = True,
                                                        labelfont   = dict  (
                                                                            size = 10,
                                                                            color = 'white',
                                                                            )
                                                        ),               

                                    colorbar    = dict  (
                                                        title       = 'System Losses (W)', 
                                                        titleside   = 'right',
                                                        titlefont   = dict  (
                                                                            size=12,
                                                                            family='Arial, sans-serif'
                                                                            )
                                                        )
                        )           )

Loss_Sys_Plot.update_layout (
                            title       = "System Losses",
                            xaxis_title = "Speed (rpm)",
                            yaxis_title = "Torque (Nm)",
                            template="plotly_white"
                            )

Loss_Sys_Plot.show()

In [ ]:
# Plot Contour of motor losses before interpolation
Eff_Inv_Plot = go.Figure(data  = go.Contour(
                                                z            = Eff_Inv_Table,
                                                x            = Speed_BP, 
                                                y            = Torque_BP,
    
                                               hovertemplate =  'Speed: %{x:.2f} rpm' + 
                                                                '<br>Torque: %{y:.2f} Nm</br>' +
                                                                'Inverter Efficiency: %{z:.2f} %',
    
                                                contours     = dict(
                                                                    coloring    ='heatmap',
                                                                    showlabels  = True,
                                                                    labelfont   = dict  (
                                                                                        size = 10,
                                                                                        color = 'white',
                                                                                        )               
                                                                    ),

                                                colorbar     = dict(
                                                                    title       = 'Inverter Efficiency (%)', 
                                                                    titleside   = 'right',
                                                                    titlefont   = dict  (
                                                                                        size=12,
                                                                                        family='Arial, sans-serif'
                                                                                        )
                                                                    )
                        )                   )

Eff_Inv_Plot.update_layout (
                            title       = "Inverter Efficiency",
                            xaxis_title = "Speed (rpm)",
                            yaxis_title = "Torque (Nm)",
                            template="plotly_white"
                            )

Eff_Inv_Plot.show()


In [ ]:
# Plot Contour of motor losses before interpolation
Eff_Mtr_Plot = go.Figure(data  = go.Contour(
                                                z           = Eff_Mtr_Table,
                                                x           = Speed_BP, 
                                                y           = Torque_BP,
    
                                                hovertemplate =  'Speed: %{x:.2f} rpm' + 
                                                                '<br>Torque: %{y:.2f} Nm</br>' +
                                                                'Motor Efficiency: %{z:.2f} %',
    
                                                contours    = dict(
                                                                    coloring    ='heatmap',
                                                                    showlabels  = True,
                                                                    labelfont   = dict  (
                                                                                        size = 10,
                                                                                        color = 'white',
                                                                                        )               
                                                                    ),

                                                colorbar    = dict(
                                                                    title       = 'Motor Efficiency (%)', 
                                                                    titleside   = 'right',
                                                                    titlefont   = dict  (
                                                                                        size=12,
                                                                                        family='Arial, sans-serif'
                                                                                        )
                                                                    )
                        )                   )

Eff_Mtr_Plot.update_layout (
                            title       = "Motor Efficiency",
                            xaxis_title = "Speed (rpm)",
                            yaxis_title = "Torque (Nm)",
                            template="plotly_white"
                            )

Eff_Mtr_Plot.show()